In [ ]:
# pip install trafilatura
# pip install summa
# pip install git+https://github.com/smirnov-am/pytopicrank.git#egg=pytopicrank
# pip install git+https://github.com/LIAAD/yake
# pip install keyBERT

In [14]:
from itertools import islice
from tqdm.notebook import tqdm
from re import sub
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from numpy import array, log


vectorizer = CountVectorizer()


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91991\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91991\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\91991\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#### Dataset loading

In [3]:
import trafilatura
array_links = [
    "https://en.wikipedia.org/wiki/Coronavirus_disease_2019", 
    "https://en.wikipedia.org/wiki/Recession", 
    "https://en.wikipedia.org/wiki/Vienna", 
    "https://en.wikipedia.org/wiki/Machine_learning", 
    "https://en.wikipedia.org/wiki/Graph_database"
]

array_text = []
for l in array_links:
    html = trafilatura.fetch_url(l)
    text = trafilatura.extract(html)
    text_clean = text.replace("\n", " ").replace("\'", "")
    array_text.append(text_clean[0:5000])

In [4]:
num_lines = sum(1 for line in open("data/wiki_tfidf_terms.csv", encoding='utf-8'))
num_lines

10394041

In [5]:
wiki_tfidf_df = pd.read_csv("data/wiki_tfidf_terms.csv")
wiki_tfidf_df.head(2)

,token,frequency,total,idf
0,the,5457533,5989879,0.093074
1,in,5234387,5989879,0.134821


#### Pre-computed IDF stats over all EN Wiki articles.  
Source: 5989879 articles from EN Wikipedia Dump 2020.01.01  
The dataset consists of gzip'ed CSV files, each is about 500 MB unpacked

* Terms (about 10M lines)
* Stems (about 10M lines)

In [6]:
num_lines = sum(1 for line in open("data/wiki_tfidf_terms.csv", encoding='utf-8'))

with open("data/wiki_tfidf_terms.csv", encoding='utf-8') as file:
    dict_idf = {}
    with tqdm(total=num_lines) as pbar:
        for i, line in tqdm(islice(enumerate(file), 1, None)):
            try: 
                cells = line.split(",")
                idf = float(sub("[^0-9.]", "", cells[3]))
                dict_idf[cells[0]] = idf
            except: 
                print("Error on: " + line)
            finally:
                pbar.update(1)

Error on: "−7,3,13,23..",1,5989879,15.605581769553712

Error on: "2,4,6,8..",1,5989879,15.605581769553712

Error on: "১,৭৫,৫৭,৩৪৫",1,5989879,15.605581769553712

Error on: ",3f.d.m.,3s.h.p.d.,8s.h.p.d",1,5989879,15.605581769553712





In [13]:
# Maximum IDF score
print("Maximum IDF score:", max(list(dict_idf.values())))
# Minimum IDF score
print("Minimum IDF score:", min(list(dict_idf.values())))

Maximum IDF score: 20062011.0
Minimum IDF score: 0.0


#### For each article, compute the TF score of its words

In [15]:
tf = vectorizer.fit_transform([x.lower() for x in array_text])
tf = tf.toarray()
tf = log(tf + 1)

In [17]:
tf.shape

(5, 1394)

Length document is 5  
Number of unique words is 1394

#### multiply TF with IDF

In [20]:
tfidf = tf.copy()
words = array(vectorizer.get_feature_names())


In [27]:
for k in tqdm(dict_idf.keys()):
    if k in words:
        tfidf[:, words == k] = tfidf[:, words == k] * dict_idf[k]
    pbar.update(1)

In [26]:
tfidf[:,:]

array([[0.        , 0.        , 0.69314718, ..., 0.        , 0.69314718,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.69314718, 0.        ,
        0.        ],
       [0.69314718, 1.09861229, 0.        , ..., 0.        , 0.        ,
        0.69314718],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

#### Print the extracted keywords

In [29]:
for j in range(tfidf.shape[0]):
    print("Keywords of article", str(j+1), words[tfidf[j, :].argsort()[-5:][::-1]])

Keywords of article 1 ['covid' 'coronavirus' 'ncov' 'cov' 'symptoms']
Keywords of article 2 ['recessions' 'recession' 'nber' 'gdp' 'shaped']
Keywords of article 3 ['vienna' 'monocles' 'km2' 'austro' 'sq']
Keywords of article 4 ['learning' 'machine' 'algorithms' 'tasks' 'algorithm']
Keywords of article 5 ['graph' 'databases' 'relational' 'database' 'nosql']
